# Работа 1.4.1. Изучение колебаний физического маятника
---
Образец обработки экспериментальных данных
----------------------------------------

Данный документ представляет собой пример обработки результатов измерений и построения графиков с помощью библиотек языка Python. В работе измеряется период колебаний физического маятника в форме сплошного стержня. Изучается зависимость периода от расстояния до точки подвеса. По измеренной зависимости проверяется теоретическая модель и вычисляется ускорение свободного падения.

In [29]:
// импорт библиотек
// для построения графиков
%use plotly
// Математические функции
@file:DependsOn("org.apache.commons:commons-math3:3.6.1")
import kotlin.math.*

## Определение погрешности измерения времени. Выбор оптимального времени измерений.

Измерим время \$n = 20$ колебаний маятника при некотором фиксированном положении подвесной призмы. Повторим измерения 8 раз и вычислим среднеквадратичное отклонение.

Результаты предварительных измерений:

In [2]:
val t0 = listOf(30.55, 30.78, 30.75, 30.64, 30.57, 30.71, 30.51, 30.65) // время 20 колебаний, [с]

Вычисляем случайную погрешность как среднеквадратичое отклонение результатов от среднего:
\begin{equation}
\sigma_t = \sqrt{\frac{1}{N-1} \sum (t_i - \bar{t})^2}
\end{equation}

In [3]:
val t_mean = t0.average() // среднее
val N = t0.size // число опытов
val sigma_t = sqrt( t0.map{t0 -> (t0 - t_mean).pow(2)}.sum() / (N - 1))
println("t_mean = ${"%.3f".format(t_mean)} ± ${"%.3f".format(sigma_t)}")

t_mean = 30.645 ± 0.097


Таким образом, случайная погрешность измерения полного времени составила $\sigma_t = 0{,}10$ с (инструментальная погрешность секундомера 0,01 c и ей можно пренебречь по сравнению с $\sigma_t$). 

Будем считать, что основной источник погрешности измерения времени -- реакция экспериментатора. Предположим, что задержка реакции человека практически  зависит от полного времени измерения. Тогда исходя требуемой для опыта относительной точности $\varepsilon = 0{,}5$%, вычислим необходимое полное время измерений:
\begin{equation}
t = \frac{\sigma_t}{\varepsilon} 
\approx 20\;с
\end{equation}

Таким образом, при периоде $T \sim1{,}5$ с достаточно провести измерение времени $n = \frac{t}{T} \sim 13$ колебаний.

## Измерение зависимости периода колебаний от положения подвесной призмы

Проводим измерение времени $t$ для $n$ колебаний для различных расстояний $a$ (расстояние от подвесной призмы до центра масс стержня).

### Данные эксперимента

Длина стержня:

In [7]:
val l = 1.000 // м

Расстояния до подвеса (от центра масс стержня):

In [8]:
val a = listOf(460.0, 420.0, 381.0, 360.0, 320.0, 291.0, 250.0, 210.0,
                180.0, 140.0, 100.0, 70.0) // [мм]
    .map { it/1e3} // перевод в [м]
val N = a.size // число точек

Количество измеренных колебаний:

In [9]:
val n = listOf(15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 10, 10)

Измеренное время, [с]:

In [10]:
val t = arrayOf(24.16, 23.63, 23.30, 23.10, 23.05, 22.95, 
                23.01, 23.44, 24.15, 25.88, 19.44, 22.69)

Периоды колебаний, [с]:

In [11]:
val T = (t zip n).map{ it.first / it.second }
println(T)

[1.6106666666666667, 1.5753333333333333, 1.5533333333333335, 1.54, 1.5366666666666666, 1.53, 1.534, 1.5626666666666666, 1.6099999999999999, 1.7253333333333332, 1.9440000000000002, 2.269]


### Оценка погрешностей исходных данных

Погрешности измерений: для длины $\sigma_a = 0{,}5$ мм (половина цены деления линейки), для периода $\sigma_T = \sigma_t/t\cdot T=\sigma_t / n$

In [12]:
val sigma_a = 0.5e-3
val sigma_T = (t zip T).map{(sigma_t / it.first)*it.second}
println(sigma_T)

[0.00649297544726068, 0.00649297544726068, 0.00649297544726068, 0.006492975447260679, 0.006492975447260679, 0.00649297544726068, 0.006492975447260679, 0.006492975447260679, 0.00649297544726068, 0.006492975447260679, 0.009739463170891019, 0.00973946317089102]


### Предварительное вычисление g

Если предположить, что справедлива теоретическая зависимость
\begin{equation}
T = 2\pi \sqrt{ \frac{l^2/12 + a^2}{g a} },
\end{equation}
то ускорение свободного падения можно найти непосредственно по данной формуле. Выразим отсюда g:

In [13]:
val gs = (a zip T).map{
    val (a, T)  = it
    4 * PI.pow(2)*(l.pow(2) / 12 + a.pow(2))/(a*(T.pow(2)))
}
val gm = gs.average()
println(gs)
println("g_mean = %.3f".format(gm))

[9.756957707213001, 9.83769174718575, 9.812527143913139, 9.845991284858295, 9.703764990789717, 9.737113203664059, 9.786464193566072, 9.810492016652528, 9.792508142427707, 9.750832857376178, 9.749986001403565, 9.665523732304052]
g_mean = 9.771


Случайную погрешность среднего найдём как стандартное отклонение, делённое на $\sqrt{N}$:

In [30]:
import org.apache.commons.math3.stat.descriptive.moment.StandardDeviation
val std = StandardDeviation()
std.incrementAll(gs.toDoubleArray())
val sigma_gm = std.getResult() / sqrt(N.toDouble())
println("sigma_gm = %.3f".format(sigma_gm))

sigma_gm = 0.016


Окончательный результат по данному методу:
\begin{equation}
\bar{g} = 9{,}771 \pm 0,015\; м/с^2
\end{equation}

Заметим, что данный результат _не может считаться надёжным_, поскольку справедливость теоретической зависимости ещё не была проверена. Кроме того, усреднение _разнородных_ измерений (при разных значениях параметра $a$) в общем случае **не является корректным**.

### Построение графика $T(a)$ и определение минимума периода
Построим график зависимости периода колебаний от положений призмы $T(a)$

In [15]:
Plotly.page{ renderer -> 
    val plotConfig = PlotlyConfig{
        responsive = true
        imageFormat = "svg"
    }
    plot("graph_1", renderer = renderer){
        // точки с погрешностями
        scatter {
            x.set(a)
            y.set(T)
            name = "Экспериментальные точки" 
            error_x{
                value = sigma_a
            }
            error_y{
                array = sigma_T
            }
            marker{
                color("black")
                size = 0
            }
            configure{
                "mode" put "markers"
            }
            
        }
        // интерполяция
        scatter {
            x.set(a)
            y.set(T)            
            name = "Кусочно линейная интерполяция" 
            line{
                color("red")
                dash = Dash.dash
            }
            configure{
                "mode" put "lines"
            }
        }
        
        scatter {
            x.set(listOf(0.00,0.5))
            y.set(listOf(1.53, 1.53))
            name = "Минимум" 
            configure{
                "mode" put "lines"
            }
        }
        // минимум
        layout{
            title{ // заголовок
                text = "\$\\text{Рис.1. График зависимости периода}~T~\\text{от положения призмы}~a\$"
            } 
            width = 700
            height = 700
            // подписи к осям
            xaxis{
                title = "\$a,~\\text{м}\$"
            }
            yaxis{
                title = "\$T,~\\text{с}\$"
            }
            
        }
    }
}


Plotly.kt

По графику визуально определяем приблизительное положение минимума: $T_{min} \approx 1{,}53 \pm 0{,}2$ с, $a_{min} \approx 280 \pm 30$ мм.

Теоретическое значение минимума: $a_{min} = \frac{l}{\sqrt{12}} \approx 289$ мм (где $l = 1000$ мм -- длина стержня) совпадает с экспериментальным в пределах погрешности.

## Построение линеаризованного графика и определение ускорения свободного падения

Согласно теории, зависимость периода от положения призмы может быть представлена в виде
\begin{equation}
T^2 a = \frac{4\pi^2}{g} \left(\frac{l^2}{12} + a^2\right).
\end{equation}

Построим график в координатах $u(v)$, где $u= T^2 a$ и $v = a^2$. Тогда теоретическая зависимость примет вид
\begin{equation}
u = k v + b,\quad где \quad k = \frac{4\pi^2}{g},\; b = \frac{\pi^2 l^2}{3g}.
\end{equation}

In [16]:
val u = (T zip a).map{it.first.pow(2) * it.second}
val v = a.map{it.pow(2)}
println("u = ${u}")
println("v = ${v}")

u = [1.1933536711111112, 1.0423035466666666, 0.9192937333333335, 0.853776, 0.7556302222222222, 0.6812018999999999, 0.5882890000000001, 0.5128046933333333, 0.4665779999999999, 0.41674851555555553, 0.37791360000000007, 0.36038527000000004]
v = [0.2116, 0.17639999999999997, 0.145161, 0.1296, 0.1024, 0.08468099999999999, 0.0625, 0.04409999999999999, 0.0324, 0.019600000000000003, 0.010000000000000002, 0.004900000000000001]


### Оценка погрешностей точек

Предварительно рассчитаем погрешности каждой точки. Согласно формулам расчёта косвенных погрешностей, имеем $\sigma_u/u = \sqrt{(2\sigma_T/T)^2 + (\sigma_a/a)^2}$, $\sigma_v = 2 a \sigma_a$:

In [17]:
val sigma_u = List(u.size){ i ->
    u[i]*sqrt(4*(sigma_T[i]/T[i]).pow(2) + (sigma_a/a[i]).pow(2))
}
val sigma_v = a.map{2 * it * sigma_a}

Вычислим относительные погрешности и убедимся, что все они малы:

In [18]:
println((sigma_u zip u).map{it.first/it.second}) // относительные погрешности
println((sigma_v zip v).map{it.first/it.second}) 

[0.008135409585101026, 0.008328822686804814, 0.008462430637464852, 0.008546050740904886, 0.0085939628535251, 0.008659720226366206, 0.008698465290538103, 0.00864448173955884, 0.008530727297310518, 0.008330984765255744, 0.011198253333507319, 0.011167780163563767]
[0.002173913043478261, 0.0023809523809523816, 0.0026246719160104982, 0.002777777777777778, 0.003125, 0.003436426116838488, 0.004, 0.004761904761904763, 0.005555555555555556, 0.007142857142857143, 0.009999999999999998, 0.014285714285714285]


Видим, что относительная погрешность величины $u$ составляет от 0,8% до 1,1%, а величины $v$ --- от 0,2% до 1,4%. Наименее точные измерения соответствуют наименьшим значениям $a$.

Строим предварительный график:

In [19]:
Plotly.page{ renderer -> 
    plot("graph_2", renderer = renderer){
        scatter { 
            x.set(v)
            y.set(u)
        }
    }
}

Plotly.kt

Видно, что график действительно имеет вид, близкий к прямой линии.

Найдём коэффициенты наилучшей прямой по методу наименьших квадратов (МНК). Для простоты погрешности всех точек будем считать приблизительно одинаковыми. Тогда можно воспользоваться формулами
\begin{equation}
k = \frac{\overline{uv}- \bar{u}\bar{v}}{\bar{v^2}-\bar{v}^2}, \quad b = \bar{u} - k \bar{v}
\end{equation}

In [20]:
val mu = u.average() // средее
val mv = v.average()
val mv2 = v.map{it.pow(2)}.average() // средний квадрат
val mu2 = u.map{it.pow(2)}.average()
val muv = (u zip v).map{it.first*it.second}.average() // среднее от произведения
val k = (muv - mu * mv) / (mv2 - mv.pow(2))
val b = mu - k * mv
println("k = ${k}, b = ${b}")

k = 4.01829503342771, b = 0.33801567301035357


Изобразим на графике экспериментальные точки и полученную аппроксимирующую зависимость.

In [21]:
Plotly.page{ renderer -> 
    val plotConfig = PlotlyConfig{
        responsive = true
        imageFormat = "svg"
    }
    plot("graph_2", renderer = renderer){
        // точки с погрешностями
        scatter {
            x.set(v)
            y.set(u)
            name = "Экспериментальные точки" 
            error_x{
                array = sigma_v
            }
            error_y{
                array = sigma_u
            }
            marker{
                color("black")
                size = 0
            }
            configure{
                "mode" put "markers"
            }
            
        }
        // аппроксимация
        scatter {
            val xs = listOf(0.0, 1.0) // две точки аппроксимирующей прямой
            x.set(xs)
            y.set(xs.map { k*it + b })          
            name = "\$\\text{Линейная интерполяция}~u = %.2f v + %.2f\$".format(k,b)
            line{
                color("red")
                dash = Dash.dash
                width = 1
            }
            configure{
                "mode" put "lines"
            }
        }
        // минимум
        layout{
            title{ // заголовок
                text = "\$\\text{Рис.2. Наилучшая прямая для линеаризованной зависимости}~T(a)\$"
            } 
            width = 700
            height = 700
            // подписи к осям
            xaxis{
                title = "\$v=a^2,~\\text{м}^2\$"
                range(0.0..0.25) // масштабы осей
            }
            yaxis{
                title = "\$u=T^2 a,~\\text{с}^2 \\cdot \\text{м}\$"
                range(0.0..1.3) // масштабы осей
            }
            
        }
    }
}

Plotly.kt

Вычислим ускорение свободного падения:

In [22]:
val g = 4 * PI.pow(2) / k
println("g = %.3f".format(g))

g = 9.825


По величине свободного слагаемого проверим длину стержня: 

In [23]:
val L = (3 * b * g) /  PI.pow(2)
println("L = %.3f м".format(L))

L = 1.009 м


Видно, что она совпадает с измеренной непосредственно $l=1000$ мм с точностью 0,9% (в пределах погрешности опыта).

Вычислим случайные погрешности определения коэффициентов прямой: 
\begin{equation}
\sigma_k = \sqrt{\frac{1}{N-2} \left(\frac{\overline{y^2}-\bar{y}^2}{\overline{x^2} - \bar{x}^2}-k^2\right)},\qquad \sigma_b = \sigma_k \sqrt{\overline{x^2}}
\end{equation}
и из них --- случайную погрешность измерения $g$ и $L$.

In [24]:
val N = v.size // число точек
val sigma_k = sqrt(( (mu2 - mu.pow(2))/(mv2 - mv.pow(2)) - k.pow(2))/(N-2))
val sigma_b = sigma_k * sqrt(mv2)
val sigma_g = sigma_k / k * g
val sigma_L = L * sqrt( (sigma_b / b).pow(2) + (sigma_g / g).pow(2) )
println("sigma_k = %.3f, sigma_b = %.3f".format(sigma_k, sigma_b))
println("sigma_g = %.3f, sigma_L = %.3f".format(sigma_g, sigma_L))

sigma_k = 0.016, sigma_b = 0.002
sigma_g = 0.040, sigma_L = 0.007


Окончательный результат для ускорения свободного падения:
    \begin{equation}
    g = 9{,}82 \pm 0{,}04 \; м/с^2.
    \end{equation}
Длина стержня, определённая как параметр теоретической зависимости:
\begin{equation}
L = 1009 \pm 7\; мм.
\end{equation}

### Проверка качества аппроксимации по сумме хи-квадрат

Для проверки точности аппроксимации, вычислим сумму $\chi^2$:
\begin{equation}
\chi^2 = \sum \left(\frac{\Delta u}{\sigma_u}\right)^2
\end{equation}

In [25]:
val chi2 = List(u.size){ i ->
    ((u[i] - (k * v[i] + b)) / sigma_u[i]).pow(2)
}.sum()
val doF = N -2
println("chi_2 = ${chi2}, chi_2/doF = ${ chi2 / doF}")

chi_2 = 3.165271091628497, chi_2/doF = 0.3165271091628497


Нормированная (на число степеней свободы) величина $\chi^2/(N-2) \approx 0{,}3 < 1$ показывает, что выполненная аппроксимация является удовлетворительной (при этом оценка погрешностей по вертикальной оси несколько завышена).

Вероятность $P$ того, что в опыте обнаружено отклонение от теоретической зависимости, можно вычислить по значению "распределения хи-квадрат". Данная функция распределения реализована в пакете scipy.stats.

In [23]:
import org.apache.commons.math3.distribution.ChiSquaredDistribution
// "кумулятивная функция распределения" (cdf) задаёт вероятность того, что величина chi^2 случайно превысит заданное значение (при заданном числе степеней свободы doF)
p_value = chi2dist.cdf(chi2, doF)
println("P-value = %.1f\%".format(p_value * 100))

Line_24.jupyter-kts (1:11 - 12) Expecting an element
Line_24.jupyter-kts (1:30 - 32) Expecting an element
Line_24.jupyter-kts (1:41 - 41) Expecting an element
Line_24.jupyter-kts (2:1 - 2) Expecting an element
Line_24.jupyter-kts (2:69 - 69) Expecting an element
Line_24.jupyter-kts (2:69 - 70) Expecting an element
Line_24.jupyter-kts (2:87 - 88) Expecting an element
Line_24.jupyter-kts (2:166 - 166) Expecting an element

Таким образом, вероятность того, что в опыте обнаружено отклонение от теории, составляет $P=2{,}3\%$.

## ДОПОЛНЕНИЕ. Нелинейная аппроксимация по методу минимума хи-квадрат

_Это дополнение не является обязательной частью учебного задания. Здесь продемонстрировано, как можно обработать результаты эксперимента, не прибегая к линеаризации графика. Мы вычислим параметры эксперимента с помощью нелинейного метода минимизации суммы хи-квадрат (известный также под названием "МНК с весами"), используя функцию_ curve_fit _пакета_ scipy.optimize.

In [24]:
from scipy.optimize import curve_fit

Line_25.jupyter-kts (1:11 - 12) Expecting an element
Line_25.jupyter-kts (1:37 - 37) Expecting an element

Зададим теоретическую зависимость как функцию $T(a)$, считая величины $g$ и $l$ параметрами модели:
\begin{equation}
T(a ; g, l) = 2\pi \sqrt{\frac{l^2/12 + a^2}{g a}}
\end{equation}

In [27]:
val f = {x: Double, g: Double, l: Double ->
    2 * PI * sqrt( (l.pow(2) / 12 + x.pow(2)) / (g * x) )}

Воспользуемся функцией curve_fit для подбора оптимальных параметров. Данный метод численно минимизирует сумму 
\begin{equation}
\sum_{i=1}^N \left(\frac{T_i - T(a_i;g,l)}{\sigma_{Ti}}\right)^2
\end{equation}
Результатом является набор оптимальных параметров $(g, l)$, и "ковариационная матрица" (диагональные элементы которой равны дисперсии соответствующего параметра).

In [26]:
popt, pcov = curve_fit(f, a, T, sigma=sigma_T) # sigma задаёт погрешности каждой точки по оси ординат

Line_27.jupyter-kts (1:5 - 6) Expecting an element
Line_27.jupyter-kts (1:48 - 49) Expecting an element
Line_27.jupyter-kts (1:102 - 102) Expecting an element

In [27]:
# оптимальные параметры содержаться в массиве popt
g, l = popt
print(g, l)

Line_28.jupyter-kts (1:1 - 2) Expecting an element
Line_28.jupyter-kts (2:2 - 3) Expecting an element

In [28]:
# Случайные погрешности параметров -- это диагональные элементы "ковариационной" матрицы pcov
sigma_g, sigma_l = np.sqrt(np.diag(pcov))
print(f"{sigma_g=}", f"{sigma_l=}")

Line_29.jupyter-kts (1:1 - 2) Expecting an element
Line_29.jupyter-kts (2:8 - 9) Expecting an element
Line_29.jupyter-kts (3:8 - 8) Expecting ','
Line_29.jupyter-kts (3:23 - 23) Expecting ','

Изобразим на графике результат нелинейной аппроксимации, наложенный на экспериментальные точки.

In [ ]:
Plotly.page{ renderer -> 
    val plotConfig = PlotlyConfig{
        responsive = true
        imageFormat = "svg"
    }
    plot("graph_1", renderer = renderer){
        // точки с погрешностями
        scatter {
            x.set(a)
            y.set(T)
            name = "Экспериментальные точки" 
            error_x{
                value = sigma_a
            }
            error_y{
                array = sigma_T
            }
            marker{
                color("black")
                size = 0
            }
            configure{
                "mode" put "markers"
            }
            
        }
        // аппроксимация
        x = List(50){i -> 0.01 + i*0.01}
        scatter {
            x.set(x)
            y.set(x.map{f(it, g, l)})            
            name = "\$\\text{Нелинейная аппроксимация}~u = %.2f v + %.3f\$".format(g,l)
            line{
                color("red")
                dash = Dash.dash
            }
            configure{
                "mode" put "lines"
            }
        }
        layout{
            title{ // заголовок
                text = "\$\\text{Рис.3. Зависимость периода}~T~\\text{от положения призмы}~a:\\text{ нелинейная аппроксимация}\$"
            } 
            width = 700
            height = 700
            // подписи к осям
            xaxis{
                title = "\$a,~\\text{м}\$"
                range(0.0..0.5) // масштабы осей
            }
            yaxis{
                title = "\$T,~\\text{с}\$"
                range(1.4..2.5) // масштабы осей
            }
            
        }
    }
}

Результат нелинейной аппроксимации:
\begin{equation}
g = 9{,}84 \pm 0{,}04\; \text{м}/\text{с}^2,\qquad l = 1{,}006 \pm 0{,}003\;\text{м}
\end{equation}

Из теоретической зависимости при найденных $g$ и $l$ определим минимальное значение периода:

In [30]:
T_min = f(l**2/np.sqrt(12), g, l)
print("T_min = %.3f с" % T_min)

Line_31.jupyter-kts (1:13 - 14) Expecting an element
Line_31.jupyter-kts (1:14 - 14) Expecting ','

Вычислим сумму $\chi^2$ и её нормировку на число степеней свободы:

In [31]:
chi2 = np.sum( ( (T - f(a, g, l)) / sigma_T )**2 )
print("chi_2 = ", chi2, ", chi_2/doF = ", chi2 / (len(a)-2))
p_value = chi2dist.cdf(chi2, doF)
print("P-value = {:.1f}%".format(p_value * 100))

Line_32.jupyter-kts (1:47 - 48) Expecting an element
Line_32.jupyter-kts (1:48 - 48) Expecting ','

Поскольку $\chi^2 / (N - 2) \lesssim 1$, аппроксимация является удовлетворительной. Оценка величины погрешностей $\sigma_T$ несколько завышена.